In [ ]:
#tensorflos.js用のライブラリをダウンロード
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 1.0 MB 43.6 MB/s 
     |████████████████████████████████| 40 kB 4.6 MB/s 
     |████████████████████████████████| 511.7 MB 6.4 kB/s 
     |████████████████████████████████| 1.6 MB 61.9 MB/s 
     |████████████████████████████████| 511.7 MB 4.8 kB/s 
     |████████████████████████████████| 5.8 MB 46.2 MB/s 
     |████████████████████████████████| 438 kB 61.5 MB/s 
     |████████████████████████████████| 5.8 MB 35.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempti

In [ ]:
#インポート
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import tensorflowjs as tfjs

KeyboardInterrupt: ignored

In [ ]:
#MNISTデータダウンロード
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
#前処理レイヤー
def preprocess(image, labels):
  return tf.cast(image, tf.float32) / 255., labels

#訓練データに前処理レイヤー適用
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

#テストデータに前処理データ適用
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE) 
ds_test = ds_test.batch(1) #テストにすべてのデータを使用する
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
#モデル構築

'''
モデルはCNNを採用
'''

input = layers.Input((28,28, 1)) #このinput層によりexpand_dimsが必要に

conv2d = layers.Conv2D(32, (3,3))(input)
act = layers.Activation("relu")(conv2d)
batch = layers.BatchNormalization()(act)
pooling = layers.MaxPool2D(pool_size=(2, 2))(batch)

conv2d = layers.Conv2D(64, (3,3))(pooling)
act = layers.Activation("relu")(conv2d)
batch = layers.BatchNormalization()(act)
pooling = layers.MaxPool2D(pool_size=(2, 2))(batch)

flatten = layers.Flatten()(pooling)
dense = layers.Dense(128)(flatten)
act = layers.Activation("relu")(dense)
batch = layers.BatchNormalization()(act)
dense = layers.Dense(10)(batch)

output = layers.Activation("softmax")(dense)

model = tf.keras.Model(input, output)

model.summary()

In [ ]:
# コールバック : ログ用にtensorBoardを使用, EarlyStopping, ModelCheckpointを採用
callbacks = [tf.keras.callbacks.TensorBoard(log_dir="logs") ,  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3), tf.keras.callbacks.ModelCheckpoint(filepath="model_bbx",save_best_only=True, monitor="val_acc")]
#モデルのコンパイプ(acc)
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics="acc" ) 
#訓練開始
history = model.fit(ds_train, validation_data=ds_test,  epochs=20, callbacks=callbacks)

In [ ]:
#最高モデルを取得
bestmodel = tf.keras.models.load_model("model_bbx")

In [ ]:
#最高モデルでの精度検証
score = bestmodel.evaluate(ds_test)

In [ ]:
#tensorflow.js用にモデルをセーブ
tfjs.converters.save_keras_model(bestmodel, "MNIST_CNN_MODEL_OUTPUT")

In [ ]:
#Zipに圧縮
!zip -r MNIST_CNN_MODEL_OUTPUT.zip MNIST_CNN_MODEL_OUTPUT